In [1]:
import tensorflow as tf
import numpy as np

from ann_utils.fully_layer import FullyLayer

from ann_utils.manager import tf_global_initializer

from ann_utils.sess import TfSess

In [2]:
sess = TfSess( "", gpu = False )

In [3]:
mem_v = [[1,1,1,1,1],[3,3,3,3,3],[5,5,5,5,5]]
ctx_v = [[5,5,5,5,5],[3,3,3,3,3]]

In [4]:
global_step = tf.compat.v1.get_variable( 'global_step', [], 
                                         initializer = tf.constant_initializer(10000), trainable = False )
decay_memory = tf.compat.v1.train.exponential_decay( 0.99, global_step, 100, 0.96, staircase = False )

# Retrive Memory

In [5]:
mem = tf.placeholder( 'float', [ 3, 5 ], name = "mem" )
ctx = tf.placeholder( 'float', [ None, 5 ], name = "ctx" )

batch_size = tf.shape(ctx)[0]
mem_size = tf.shape(mem)[0]

In [6]:
norm_mem = tf.norm( mem, axis = 1, keepdims = True )
norm_ctx = tf.norm( ctx, axis = 1, keepdims = True )

In [7]:
print(ctx)
print(mem)
print(norm_ctx)
print(norm_mem)

Tensor("ctx:0", shape=(?, 5), dtype=float32)
Tensor("mem:0", shape=(3, 5), dtype=float32)
Tensor("norm_1/Sqrt:0", shape=(?, 1), dtype=float32)
Tensor("norm/Sqrt:0", shape=(3, 1), dtype=float32)


In [8]:
simi = 1.0 - tf.nn.l2_normalize( tf.sqrt( tf.square( norm_ctx - tf.transpose( norm_mem ) ) ), axis = 1 )
simi

<tf.Tensor 'sub_1:0' shape=(?, 3) dtype=float32>

In [9]:
scores = tf.nn.softmax( simi )
scores

<tf.Tensor 'Softmax:0' shape=(?, 3) dtype=float32>

In [10]:
print( sess( norm_ctx, { mem: mem_v, ctx: ctx_v } ) )
print("\n")
print( sess( norm_mem, { mem: mem_v, ctx: ctx_v } ) )
print("\n")
print( sess( simi, { mem: mem_v, ctx: ctx_v } ) )
print("\n")
print( sess( scores, { mem: mem_v, ctx: ctx_v } ) )
print("\n")

[[11.18034 ]
 [ 6.708204]]


[[ 2.236068]
 [ 6.708204]
 [11.18034 ]]


[[0.10557276 0.55278635 1.        ]
 [0.29289323 1.         0.29289317]]


[[0.19960546 0.31217262 0.48822188]
 [0.24825509 0.50348985 0.24825507]]




In [11]:
retrived = tf.matmul( scores, mem )
retrived

<tf.Tensor 'MatMul:0' shape=(?, 5) dtype=float32>

In [12]:
sess( [ scores, retrived ], { mem: mem_v, ctx: ctx_v } )

[array([[0.19960546, 0.31217262, 0.48822188],
        [0.24825509, 0.50348985, 0.24825507]], dtype=float32),
 array([[3.5772328, 3.5772328, 3.5772328, 3.5772328, 3.5772328],
        [3.       , 3.       , 3.       , 3.       , 3.       ]],
       dtype=float32)]

# Write Memory

In [13]:
to_write = tf.tile( ctx, tf.stack( [ 1, mem_size ] ) )
to_write = tf.reshape( to_write, [ batch_size, mem_size, 5 ] )
to_write

<tf.Tensor 'Reshape:0' shape=(?, 3, 5) dtype=float32>

In [18]:
sess( scores >= tf.reduce_max( scores, axis = 1, keepdims = True ), { mem: mem_v, ctx: ctx_v }  )

array([[False, False,  True],
       [False,  True, False]])

In [ ]:
expand_scores = tf.tile( scores, [ 1, 5 ] )
new_scores = tf.reshape( expand_scores, [ batch_size, 5, 3 ] )
new_scores

In [ ]:
to_write_scores = tf.transpose( new_scores, perm=[0,2,1] )
to_write_scores

In [ ]:
to_write_s = to_write_scores * to_write
to_write_s

In [ ]:
sess( to_write, { mem: mem_v, ctx: ctx_v } )

In [ ]:
sess( to_write_scores, { mem: mem_v, ctx: ctx_v } )

In [ ]:
sess( to_write_s, { mem: mem_v, ctx: ctx_v } )

In [ ]:
new_m = decay_memory * tf.reduce_mean( to_write_s, axis = 0 )
new_m

In [ ]:
old_m = ( ( 1.0 - decay_memory ) * mem )
old_m

In [ ]:
op_write = old_m + new_m
op_write

In [ ]:
tf_global_initializer( sess )

In [ ]:
print("---------------Memory------------------")
print( sess( mem, 
      { mem: mem_v, ctx: ctx_v } ) )
print("\n-------------New Memory----------------")
print( sess( new_m, 
      { mem: mem_v, ctx: ctx_v } ) )
print("\n-----------Old Memory-------------------")
print( sess( old_m, 
      { mem: mem_v, ctx: ctx_v } ) )
print("\n---------Final Memory-------------------")
print( sess( op_write, 
      { mem: mem_v, ctx: ctx_v } ) )
print("---------------------------------")